## Org MADS

In [1]:
import numpy as np

def objective(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

# Bounds for each variable
bounds = np.array([[0, 1], [10, 20], [0, 3]])

# Initial guess
x0 = np.array([0.9, 19, 0.1])

def mads(func, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
    """ Mesh Adaptive Direct Search (MADS) implementation. """
    n = len(x0)
    x = x0.copy()
    delta_min = tol
    iteration = 0
    
    while delta > delta_min and iteration < max_iter:
        improved = False
        
        # Generate trial points
        for i in range(n):
            for direction in [-1, 1]:
                x_new = x.copy()
                x_new[i] += direction * delta
                x_new = np.clip(x_new, bounds[:, 0], bounds[:, 1])  # Enforce bounds
                
                if func(x_new) < func(x):
                    x = x_new
                    improved = True
        
        # Reduce mesh size if no improvement
        if not improved:
            delta /= 2.0
        
        iteration += 1
        yield x, func(x), iteration

# Run MADS optimization
mads_iter = mads(objective, x0, bounds)
for x, fval, step in mads_iter:
    print(f"Step {step}: x = {x}, f(x) = {fval}")

Step 1: x = [ 0.9 18.   1.1], f(x) = 9.32
Step 2: x = [ 0.9 17.   1.1], f(x) = 4.32
Step 3: x = [ 0.9 16.   1.1], f(x) = 1.32
Step 4: x = [ 0.9 15.   1.1], f(x) = 0.31999999999999995
Step 5: x = [ 0.9 15.   1.1], f(x) = 0.31999999999999995
Step 6: x = [ 0.4 15.   1.6], f(x) = 0.02000000000000001
Step 7: x = [ 0.4 15.   1.6], f(x) = 0.02000000000000001
Step 8: x = [ 0.4 15.   1.6], f(x) = 0.02000000000000001
Step 9: x = [ 0.525 15.     1.475], f(x) = 0.0012499999999999968
Step 10: x = [ 0.525 15.     1.475], f(x) = 0.0012499999999999968
Step 11: x = [ 0.525 15.     1.475], f(x) = 0.0012499999999999968
Step 12: x = [ 0.49375 15.       1.50625], f(x) = 7.812500000000083e-05
Step 13: x = [ 0.49375 15.       1.50625], f(x) = 7.812500000000083e-05
Step 14: x = [ 0.49375 15.       1.50625], f(x) = 7.812500000000083e-05
Step 15: x = [ 0.5015625 15.         1.4984375], f(x) = 4.882812499999793e-06
Step 16: x = [ 0.5015625 15.         1.4984375], f(x) = 4.882812499999793e-06
Step 17: x = [ 0.501

### Stepwise MADS, on iterations

In [13]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
        self.best_value = None  # Store the best evaluation value
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates.copy()
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")
        
        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]
        
        if self.best_value is None or best_value < self.best_value:
            self.x = best_candidate
            self.best_value = best_value
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def get_current_solution(self):
        return self.x




In [14]:
# Example Usage
def obj_func(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = StepwiseMADS(x0=[0.9, 19, 0.1], bounds=bounds)

In [16]:
mads.pending_eval

In [18]:
# Example Usage
def obj_func(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = StepwiseMADS(x0=[0.9, 19, 0.1], bounds=bounds)
mads.best_value = obj_func(mads.x)  # Initialize best value with initial point evaluation
print("initial best value", mads.best_value)

for _ in range(20):  # Run for a few iterations
    candidates = mads.get_next_candidate()
    # print(mads.pending_eval)
    evaluations = [float(obj_func(c)) for c in candidates]  # Simulated evaluation
    mads.update_with_result(evaluations)
    # print(f"Iter {_}candidates, evaluations", list(zip(candidates, evaluations)))
    print(f"Iteration {_}: Best solution {mads.get_current_solution()} with value {mads.best_value}")
    

initial best value 18.12
Iteration 0: Best solution [ 0.9 18.   0.1] with value 11.12
Iteration 1: Best solution [ 0.9 16.   0.1] with value 3.12
Iteration 2: Best solution [ 0.9 16.   0.1] with value 3.12
Iteration 3: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 4: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 5: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 6: Best solution [ 0.9 15.   2.1] with value 0.5200000000000001
Iteration 7: Best solution [ 0.9 15.   2.1] with value 0.5200000000000001
Iteration 8: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 9: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 10: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 11: Best solution [ 0.4 15.   1.1] with value 0.16999999999999993
Iteration 12: Best solution [ 0.4 15.   1.1] with value 0.16999999999999993
Iteration 13: Best solution [ 0.4 15.   1.6] w

## Stp on every call

In [29]:
import math

class MADS:
    def __init__(self, x0, initial_value, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.stp_MADS = StepwiseMADS(x0, bounds=bounds, delta0=delta0, delta_min=delta_min, alpha=alpha, gamma=gamma)
        self.candidates = []
        self.evaluations = []
        self.step_num = 0
        self.stp_MADS.best_value = initial_value

    def step(self, evaluation):
        self.evaluations.append(evaluation)
        if self.candidates == []:
            # new MADS iteration
            print("no candidates...")
            
            self.candidates = self.stp_MADS.get_next_candidate()
            self.stp_MADS.update_with_result(self.evaluations)
            self.evaluations = []
        else:
            print("pending candidatas...")
        return self.candidates.pop(0)

In [30]:
x0=[0.9, 19, 0.1]
initial_value = obj_func(x0)
bounds = np.array([[0, 1], [10, 20], [0, 3]])
ab = MADS(x0, initial_value, bounds)

In [31]:
x = ab.step(initial_value)

no candidates...


In [32]:
x

array([ 1. , 19. ,  0.1])

In [37]:
for _ in range(200): 
    print(f"current x {x}")
    evaluation = obj_func(x)
    x = ab.step(evaluation)
    print(f"step {_}, {evaluation}, next x, {x}")

current x [ 0.9 18.5  0.6]
pending candidatas...
step 0, 13.22, next x, [ 0.4 18.5  0.1]
current x [ 0.4 18.5  0.1]
pending candidatas...
step 1, 14.219999999999999, next x, [ 0.9 18.   0.1]
current x [ 0.9 18.   0.1]
pending candidatas...
step 2, 11.12, next x, [ 0.9 18.5  0. ]
current x [ 0.9 18.5  0. ]
no candidates...
step 3, 14.66, next x, [ 1.  18.   0.1]
current x [ 1.  18.   0.1]
pending candidatas...
step 4, 11.209999999999999, next x, [ 0.9 19.   0.1]
current x [ 0.9 19.   0.1]
pending candidatas...
step 5, 18.12, next x, [ 0.9 18.   1.1]
current x [ 0.9 18.   1.1]
pending candidatas...
step 6, 9.32, next x, [ 0.  18.   0.1]
current x [ 0.  18.   0.1]
pending candidatas...
step 7, 11.209999999999999, next x, [ 0.9 17.   0.1]
current x [ 0.9 17.   0.1]
pending candidatas...
step 8, 6.12, next x, [ 0.9 18.   0. ]
current x [ 0.9 18.   0. ]
no candidates...
step 9, 11.41, next x, [ 1.  18.   0.1]
current x [ 1.  18.   0.1]
pending candidatas...
step 10, 11.209999999999999, next 